In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

<a target="_blank" href="https://colab.research.google.com/github/ai-safety-foundation/sparse_autoencoder/blob/main/docs/content/demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Training Demo

This is a quick start demo to get training a SAE right away. All you need to do is choose a few
hyperparameters (like the model to train on), and then set it off.

In this demo we'll train a sparse autoencoder on all MLP layer outputs in GPT-2 small (effectively
training an SAE on each layer in parallel).

## Setup

### Imports

In [2]:
# Check if we're in Colab
try:
    import google.colab  # noqa: F401 # type: ignore

    in_colab = True
except ImportError:
    in_colab = False

#  Install if in Colab
if in_colab:
    %pip install sparse_autoencoder transformer_lens transformers wandb

# Otherwise enable hot reloading in dev mode
if not in_colab:
    %load_ext autoreload
    %autoreload 2

In [3]:
import os

from sparse_autoencoder import (
    ActivationResamplerHyperparameters,
    AutoencoderHyperparameters,
    Hyperparameters,
    LossHyperparameters,
    Method,
    OptimizerHyperparameters,
    Parameter,
    PipelineHyperparameters,
    SourceDataHyperparameters,
    SourceModelHyperparameters,
    SweepConfig,
    sweep,
)

os.environ["WANDB_NOTEBOOK_NAME"] = "demo.ipynb"

### Hyperparameters

Customize any hyperparameters you want below (by default we're sweeping over l1 coefficient and
learning rate).

Note we are using the RANDOM sweep approach (try random combinations of hyperparameters), which
works surprisingly well but will need to be stopped at some point (as otherwise it will continue
forever). If you want to run pre-defined runs consider using `Parameter(values=[0.01, 0.05...])` for
example rather than `Parameter(max=0.03, min=0.008)` for each parameter you are sweeping over. You
can then set the strategy to `Method.GRID`.

In [9]:
def train_gpt_small_mlp_layers(
    expansion_factor: int = 4,
    n_layers: int = 12,
) -> None:
    """Run a new sweep experiment on GPT 2 Small's MLP layers.

    Args:
        expansion_factor: Expansion factor for the autoencoder.
        n_layers: Number of layers to train on. Max is 12.

    """
    sweep_config = SweepConfig(
        parameters=Hyperparameters(
            loss=LossHyperparameters(
                l1_coefficient=Parameter(max=0.03, min=0.008),
            ),
            optimizer=OptimizerHyperparameters(
                lr=Parameter(max=0.001, min=0.00001),
            ),
            source_model=SourceModelHyperparameters(
                name=Parameter("gpt2"),
                cache_names=Parameter(
                    [f"blocks.{layer}.hook_mlp_out" for layer in range(n_layers)]
                ),
                hook_dimension=Parameter(768),
            ),
            source_data=SourceDataHyperparameters(
                dataset_path=Parameter("alancooney/sae-monology-pile-uncopyrighted-tokenizer-gpt2"),
                context_size=Parameter(256),
                pre_tokenized=Parameter(value=True),
                pre_download=Parameter(value=False),  # Default to streaming the dataset
            ),
            autoencoder=AutoencoderHyperparameters(
                expansion_factor=Parameter(value=expansion_factor),
                type=Parameter("tanh_encoder"),
            ),
            pipeline=PipelineHyperparameters(
                max_activations=Parameter(1_000_000_000),
                checkpoint_frequency=Parameter(100_000_000),
                validation_frequency=Parameter(100_000_000),
                max_store_size=Parameter(100_000),
                source_data_batch_size=Parameter(16),
                train_batch_size=Parameter(8192)
            ),
            activation_resampler=ActivationResamplerHyperparameters(
                resample_interval=Parameter(200_000_000),
                n_activations_activity_collate=Parameter(100_000_000),
                threshold_is_dead_portion_fires=Parameter(1e-6),
                max_n_resamples=Parameter(4),
                resample_dataset_size=Parameter(81920),
            ),
        ),
        method=Method.RANDOM,
    )

    sweep(sweep_config=sweep_config)

### Run the sweep

This will start a sweep with just one agent (the current machine). If you have multiple GPUs, it
will use them automatically. Similarly it will work on Apple silicon devices by automatically using MPS.

In [5]:
os.environ["TOKENIZERS_PARALLELISM"]="false"

In [10]:
train_gpt_small_mlp_layers()

Create sweep with ID: 00f0cyw4
Sweep URL: https://wandb.ai/hufy-dev/sparse-autoencoder/sweeps/00f0cyw4


wandb: Agent Starting Run: dsm2zn3g with config:
wandb: 	activation_resampler: {'max_n_resamples': 4, 'n_activations_activity_collate': 100000000, 'resample_dataset_size': 409600, 'resample_interval': 200000000, 'threshold_is_dead_portion_fires': 1e-06}
wandb: 	autoencoder: {'expansion_factor': 4, 'type': 'tanh_encoder'}
wandb: 	loss: {'l1_coefficient': 0.02815741075080031}
wandb: 	optimizer: {'adam_beta_1': 0.9, 'adam_beta_2': 0.99, 'adam_weight_decay': 0, 'amsgrad': False, 'fused': False, 'lr': 0.0008266150398536362}
wandb: 	pipeline: {'checkpoint_frequency': 100000000, 'log_frequency': 100, 'max_activations': 1000000000, 'max_store_size': 500000, 'num_workers_data_loading': 0, 'source_data_batch_size': 8, 'train_batch_size': 4096, 'validation_frequency': 100000000, 'validation_n_activations': 8192}
wandb: 	random_seed: 49
wandb: 	source_data: {'context_size': 256, 'dataset_column_name': 'input_ids', 'dataset_path': 'alancooney/sae-monology-pile-uncopyrighted-tokenizer-gpt2', 'pre_do

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Activations trained on:   0%|          | 0/1000000000 [00:00<?, ?it/s]

/home/hufy/.cache/pypoetry/virtualenvs/sparse-autoencoder-PMXxRQv7-py3.11/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
/home/hufy/.cache/pypoetry/virtualenvs/sparse-autoencoder-PMXxRQv7-py3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:212: You called `self.log('train/neuron_activity/blocks.0.hook_mlp_out', ...)` in your `training_step` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don't intend to reduce the value (for instance when logging the global step or epoch) then you can use `self.logger.log_metrics({'train/neuron_activity/blocks.0.hook_mlp_out': ...})` ins

: 

Want to speed things up? You can trivially add extra machines to the sweep, each of which will peel
of some runs from the sweep agent (stored on Wandb). To do this, on another machine simply run:

```bash
pip install sparse_autoencoder
join-sae-sweep --id=SWEEP_ID_SHOWN_ON_WANDB
```